In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv("./nona_train.csv",nrows=1000)
train_df = train_df.drop(columns='Unnamed: 0',axis=1)

In [3]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [4]:
train_df.columns

Index(['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND', 'BROWSE_NODE_ID'], dtype='object')

In [4]:
train_df['TEXT'] = train_df['TITLE'] + train_df['DESCRIPTION'] + train_df['BULLET_POINTS'] + train_df['BRAND']

### for each term in our dataset, we will calculate a measure called Term Frequency, Inverse Document Frequency

In [5]:
y_train = train_df['BROWSE_NODE_ID']
x_train = train_df['TEXT']
xtrain, xvalid, ytrain, yvalid = train_test_split(x_train.values, y_train.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [44]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

In [46]:
tfv.fit(xtrain)
xtrain_tfv =  tfv.transform(xtrain).toarray()
xtrain_tfv.shape

(800, 4507)

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np


In [43]:
len(tfv.get_feature_names())

5630

In [28]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)

LogisticRegression()

In [29]:
predictions = clf.predict_proba(xvalid_tfv)

In [32]:
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

IndexError: index 984 is out of bounds for axis 1 with size 565